In [1]:
# Load all the required libraries
import json
import pandas as pd
from IPython.display import display, HTML

In [2]:
def load_json_file(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

In [3]:
# Load different user schemas as json dictionaries
users_4me = load_json_file("./4me/group.schema.json")
users_sf = load_json_file("./SF/groups.json")
users_slack = load_json_file("./Slack/slack-groups-schemas.json")

In [4]:
users_4me_attribute_list = users_4me["attributes"]
users_sf_attribute_list = users_sf["attributes"]
users_slack_attribute_list = users_slack["attributes"]

In [5]:
def get_attribute_name(attribute_list):
    attribute_name_list = []
    for attribute in attribute_list:
        attribute_name_list.append(attribute["name"])
    return sorted(attribute_name_list)

In [6]:
users_4me_attribute_name_list = get_attribute_name(users_4me_attribute_list)
users_sf_attribute_name_list = get_attribute_name(users_sf_attribute_list)
users_slack_attribute_name_list = get_attribute_name(users_slack_attribute_list)

In [7]:
pd.DataFrame(
    data=[
        users_4me_attribute_name_list,
        users_sf_attribute_name_list,
        users_slack_attribute_name_list,
    ],
    index=["4me", "salesforce", "slack"],
).transpose()

,4me,salesforce,slack
0,displayName,displayName,displayName
1,groupType,id,id
2,members,members,members


In [8]:
df_4me = pd.DataFrame(data=users_4me_attribute_name_list, columns=["4me"])
df_sf = pd.DataFrame(data=users_sf_attribute_name_list, columns=["salesforce"])
df_slack = pd.DataFrame(data=users_slack_attribute_name_list, columns=["slack"])


df_merged = df_sf.merge(
    df_4me, how="outer", left_on="salesforce", right_on="4me"
).merge(df_slack, how="outer", left_on="salesforce", right_on="slack")

df_merged.style.highlight_null(color="red")

,salesforce,4me,slack
0,displayName,displayName,displayName
1,id,nan,id
2,members,members,members
3,nan,groupType,nan


In [10]:
common_elements = (
    set(users_4me_attribute_name_list)
    & set(users_sf_attribute_name_list)
    & set(users_slack_attribute_name_list)
)
common_elements = sorted(common_elements)

for common_element in common_elements:
    for attribute in users_4me_attribute_list:
        if attribute["name"] == common_element:
            users_4me_attribute = attribute
    for attribute in users_sf_attribute_list:
        if attribute["name"] == common_element:
            users_sf_attribute = attribute
    for attribute in users_slack_attribute_list:
        if attribute["name"] == common_element:
            users_slack_attribute = attribute
    print("=" * 100)
    print(f"Attribute: {common_element}")
    pd.set_option("display.max_colwidth", 30)
    pd.set_option("display.width", 1000)
    df_4me = pd.DataFrame(
        {"attribute": users_4me_attribute.keys(), "value": users_4me_attribute.values()}
    )
    df_sf = pd.DataFrame(
        {"attribute": users_sf_attribute.keys(), "value": users_sf_attribute.values()}
    )
    df_slack = pd.DataFrame(
        {
            "attribute": users_slack_attribute.keys(),
            "value": users_slack_attribute.values(),
        }
    )
    df = df_4me.merge(
        df_sf, how="outer", on="attribute", suffixes=("_salesforce", "_salesforce")
    ).merge(df_slack, how="outer", on="attribute", suffixes=("_slack", "_"))
    df = df.set_index("attribute")
    df.style.highlight_null(color="yellow")
    display(HTML(df.to_html()))

Attribute: displayName


,value_salesforce,value_salesforce,value
attribute,,,
caseExact,False,False,False
description,A human-readable name for the Group.,"The name of the User, suitable for display",A human readable name for the Group. REQUIRED.
multiValued,False,False,False
mutability,readWrite,readWrite,readWrite
name,displayName,displayName,displayName
required,True,False,True
returned,default,NaN,always
type,string,string,string
uniqueness,server,NaN,server


Attribute: members


,value_salesforce,value_salesforce,value
attribute,,,
caseExact,NaN,NaN,False
description,A list of members of the Group.,The members of the group,"A list of members of the Group. Canonical Type ""User"" is currently supported and READ-ONLY. The value must be the ""id"" of a SCIM User resource."
multiValued,True,True,True
multiValuedAttributeChildName,NaN,NaN,member
mutability,readWrite,readWrite,readWrite
name,members,members,members
required,False,False,True
returned,default,NaN,default
subAttributes,"[{'name': 'value', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'immutable', 'returned': 'default', 'uniqueness': 'none', 'description': 'Identifier of the member of this Group.'}, {'name': '$ref', 'type': 'reference', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'immutable', 'returned': 'default', 'uniqueness': 'none', 'referenceTypes': ['Group', 'User'], 'description': 'The URI corresponding to a SCIM resource that is a member of this Group.'}, {'name': 'type', 'type': 'string', 'multiValued': False, 'required': False, 'caseExact': False, 'mutability': 'immutable', 'returned': 'default', 'uniqueness': 'none', 'canonicalValues': ['Group', 'User'], 'description': 'A label indicating the type of resource, e.g., 'User' or 'Group'.'}]","[{'name': 'value', 'type': 'string', 'description': 'The id of the member', 'mutability': 'readWrite', 'required': False, 'caseExact': False}, {'name': 'type', 'type': 'string', 'description': 'The type of membership of the member in the group', 'mutability': 'readOnly', 'required': True, 'caseExact': False}]","[{'name': 'value', 'type': 'string', 'multiValued': False, 'description': 'The ""id"" of a SCIM User resource. REQUIRED.', 'mutability': 'immutable', 'returned': 'default', 'uniqueness': 'none', 'required': True, 'caseExact': False}, {'name': 'display', 'type': 'string', 'multiValued': False, 'description': 'A human readable name for the member, primarily used for display purposes. READ-ONLY.', 'mutability': 'immutable', 'returned': 'default', 'uniqueness': 'none', 'required': False, 'caseExact': False}]"
